In [20]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()

In [4]:
load_dotenv(override=True)


True

In [5]:
instructions1 = "You are a sales agent working for Coupa, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for Coupa, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for Coupa, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [6]:
sales_agent1 = Agent(name="sales_agent1", instructions=instructions1, model="gpt-4o-mini")
sales_agent2 = Agent(name="sales_agent2", instructions=instructions2, model="gpt-4o-mini")
sales_agent3 = Agent(name="sales_agent3", instructions=instructions3, model="gpt-4o-mini")







In [8]:
result = Runner.run_streamed(sales_agent1, "Write a cold email to a potential customer")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print (event.data.delta, end="", flush=True)


Subject: Enhancing Your SOC 2 Compliance Process with Coupa

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am reaching out on behalf of Coupa, where we specialize in simplifying SOC 2 compliance through our innovative SaaS solution.

In today’s regulatory landscape, ensuring data security and compliance is more critical than ever. Our AI-powered platform streamlines the entire SOC 2 preparation and audit process, allowing organizations like yours to minimize risks, reduce manual efforts, and simplify the management of compliance documentation.

Some key benefits of our solution include:

- **Automated Evidence Collection**: Easily gather and manage necessary documentation without the usual stress of manual tracking.
- **Real-time Compliance Monitoring**: Stay ahead of compliance requirements and audits with continuous monitoring and alerts.
- **User-Friendly Interface**: Our intuitive platform ensures that your team can quickly adapt, savin

In [10]:
message = "Write a cold email to a potential customer"

with trace("Parallel Cold Emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
outputs = [result.final_output for result in results]
for output in outputs:
    print (output + "\n\n")


Subject: Simplify Your SOC 2 Compliance with AI-Powered Solutions

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out on behalf of Coupa, where we specialize in helping organizations like yours navigate the complexities of SOC 2 compliance.

As you may know, maintaining SOC 2 compliance can be a daunting task—requiring time, resources, and meticulous attention to detail. Our AI-powered SaaS tool is designed to streamline this process, making it easier for your team to prepare for audits and minimize risks. 

With Coupa, you can:

- Automate documentation and evidence collection
- Gain real-time insights into your compliance posture
- Simplify audit management and reporting 

I would love to schedule a brief call to discuss how our solution can help enhance your compliance strategy and support your business goals. Please let me know a convenient time for you, and I’ll be happy to arrange it.

Thank you for considering Coupa as your

In [11]:
#Sales picker

sales_picker = Agent(name="sales_picker", instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.", model="gpt-4o-mini")



In [15]:
message = "write a cold sales email"

with trace("Selection email for final round"):
    result = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold emails: \n \n:".join(outputs)

    best = await Runner.run(sales_picker, emails)
    print (f'Best Sales Email: \n{best.final_output}')




Best Sales Email: 
Subject: Let’s Make SOC2 Compliance Less of a Headache! 🤕➡️💊

Hi [First Name],

I hope this email finds you in good spirits and not buried under a mountain of documents! If you are, I totally understand. SOC2 compliance can feel like a never-ending episode of a reality show. “Survivor: Audit Edition,” anyone?

At Coupa, we believe that dealing with compliance shouldn’t require a white flag or a designated “audit survival kit.” 🏴‍☠️ That’s why we’ve cooked up an AI-powered SaaS tool that turns audits from terrifying dragons into friendly puppies. (Well, sort of—there's still some paperwork involved, but we keep the growling to a minimum!)

With our tool, you’ll:

🐕 Wag your tail at faster prep times  
🚀 Launch into a stress-free audit phase  
🎩 Turn compliance into a magic trick—now you see it, now you don’t!

Let’s schedule a quick chat to see how we can keep those pesky audits from taking all your time and sanity. How does your calendar look this week?

Looking forw

In [28]:
@function_tool
def send_email(body:str):
    '''Send out an email with given body to all sales prospects'''
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENTGRID_API_KEY"))
    from_email = Email("samson.work@gmail.com")
    to_email = To("samjosoff@gmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales Email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status":"success"}


In [27]:
send_email("Hello, this is a test email")

{'status': 'success'}

In [30]:
send_email

FunctionTool(name='send_email', description='Send out an email with given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002575CFEE8E0>, strict_json_schema=True)

In [40]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [42]:
email_tool_description = "write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name = "sales_agent1", tool_description = email_tool_description)
tool2 = sales_agent2.as_tool(tool_name = "sales_agent2",tool_description=email_tool_description )
tool3 = sales_agent3.as_tool(tool_name = 'sales_agent3', tool_description = email_tool_description)
tools = [tool1, tool2, tool3, send_email]
tools

[FunctionTool(name='sales_agent1', description='write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002575F2ED260>, strict_json_schema=True),
 FunctionTool(name='sales_agent2', description='write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002575F2EDA80>, strict_json_schema=True),
 FunctionTool(name='sales_agent3', description='write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title'

# Sales Manager

In [43]:
instructions ="You are a sales manager working for Coupa. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


In [44]:
sales_manager = Agent(name='Sales_manager', instructions=instructions, tools=tools, model='gpt-4o-mini')

message = "Send a Cold Sales Email addressed as 'Dear CEO'"

with trace("Sales Manager"):
    result = await Runner.run(sales_manager, message)

In [45]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

In [48]:
subject_writer = Agent(name="subject_writer", instructions=subject_instructions, model="gpt-4o-mini")
html_writer = Agent(name="html_writer", instructions=html_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")
html_tool = html_writer.as_tool(tool_name="html_writer", tool_description="Convert a text email body to an HTML email body")



In [49]:
@function_tool
def send_html_email(subject:str, html_body:str)->dict [str, str]:
    '''Send out an HTML email with given subject and body to all sales prospects'''
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENTGRID_API_KEY"))
    from_email = Email("samson.work@gmail.com")
    to_email = To("samjosoff@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status":"success"}


In [51]:
email_sender_tools = [subject_tool, html_tool, send_html_email]
email_sender_tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002575F328D60>, strict_json_schema=True),
 FunctionTool(name='html_writer', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002575F328EA0>, strict_json_schema=True),
 FunctionTool(name='send_html_email', description='Send out an HTML email with given subject and body to all sales prospects', params_json_schema={'p

In [52]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

emailer_agent = Agent(name="emailer_agent", instructions=instructions, tools=email_sender_tools, 
                      model="gpt-4o-mini", handoff_description="Convert an email to HTML and send it")





In [54]:
handoffs = [emailer_agent]

In [56]:
sales_manager_instructions = "You are a sales manager working for Coupa. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."

sales_manager = Agent(name='Sales_manager', instructions=sales_manager_instructions, 
                      tools=tools, model='gpt-4o-mini', handoffs=handoffs)

message = "Send a Cold Sales Email addressed as to Dear CEO from Samson"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)
